<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# XLNet

BERT es uno de los modelos transformers más potentes y versátiles, que puede ser aplicado a una amplia variedad de tareas de PLN con bueno resultados. 
Sin embargo, BERT tiene dos importantes limitaciones:


1. Una de las estrategias que utiliza BERT para su entrenamiento es enmascarar tokens de la entrada para generar los embeddings. No sabemos cómo BERT utiliza esos tokens enmascarados. 
2. Si en una oración, hay más de dos tokens enmascarados, las dependencias entre estos tokens no pueden ser capturadas, así que podemos perder información importante de cada uno de ellos. 
3. La entrada de BERT está limitada a 512 tokens. 

En este notebook, presentaremos un nuevo modelo transformer, **XLNet**, que trata de superar las limitaciones de BERT, al capturar el contexto alrededor de un token aplicando un modelo de lenguaje basado en permutaciones. 

<center>
<img src='https://production-media.paperswithcode.com/methods/0_dwpaEuNcwmPQUOKi_COw4cmZ.png'>
</center>


Review this tutorial: 
- https://towardsdatascience.com/what-is-xlnet-and-why-it-outperforms-bert-8d8fce710335


##¿Qué es XLNet?

Es un modelo auto-regresivo, es decir, está entrenado para predecir la siguiente palabra a partir de una secuencia de palabras. XLNet utiliza un modelo de lenguaje basado en permutaciones para aprender representaciones de las palabras que sean capaces de capturar su contexto en ambas direcciones.

A diferencia de BERT, XLNet no utiliza ninguna estrategia de enmascaramiento de palabras. En su lugar, realiza permutaciones para capturar el contexto de una palabra, entrenando un modelo auto-regresivo a partir de todas las posibles permutaciones de las palabras en un oración. 

De esta forma para cada token, XLNet utiliza toda la información contextual del resto de tokens en la oración. Esto permite va a permitir obtener buenas representaciones de las palabras. 

Como muchos otros modelos transformers, XLNet añada una capa línea después de su modelo autoregresivo para que pueda ser adaptado fácilmente a otras tareas. 

XLNet es uno de los pocos modelos que no limitan el tamaño de la secuencia de entrada. 


## Cómo utilizar XLNet desde HuggingFace

Además de instalar la librería transformers, también necesitamos instalar **sentencepiece**, una re-implementación del tokenizador para ciertos transformers como T5 o XLNet. 



In [4]:
!pip install transformers sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00


In [5]:
from transformers import XLNetTokenizer, XLNetForMultipleChoice
import torch




### XLNet para Queation Answering.

Una de las principales ventajas de HuggingFace es que ya proporciona modelos previamente entrenados y ajustados para tareas específicas. En este caso, vamos a utilizar la clase **XLNetForMultipleChoice** que ya encapsula el modelo ajustado para la tarea de encontrar la respuesta correcta a una pregunta entre múltiples respuestas. 

In [6]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
model = XLNetForMultipleChoice.from_pretrained("xlnet-base-cased", return_dict = True).to("cuda")



Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultipleChoice: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


El modelo toma como entrada una pregunta y una serie de posibles respuestas. Entonces, va a computar la probabilidad de cada respuesta. Para hacer esto, utiliza la función **softmax** sobre la salida de XLNet. El valor más alto será la respuesta correcta. 



In [15]:
from torch.nn import functional as F

prompt = "What is the capital of Spain?"
answers = ["Paris", "London", "Lyon", "Berlin", "Madrid"]

encoding = tokenizer([prompt, prompt, prompt, prompt, prompt], answers, return_tensors="pt", padding = True).to("cuda")

outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}) 

logits = outputs.logits
softmax = F.softmax(logits, dim = -1)
index = torch.argmax(softmax, dim = -1)
print("The correct answer is", answers[index])

tensor([[0.1879, 0.2055, 0.1661, 0.2092, 0.2312]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
The correct answer is Madrid


Vamos a mostrar la probabilidad que el modelo obtuvó para cada una de las posibles respuestas: 

In [ ]:
for i in range(len(answers)):
    print(answers[i], softmax[0][i])


Paris tensor(0.2096, device='cuda:0', grad_fn=<SelectBackward0>)
London tensor(0.2278, device='cuda:0', grad_fn=<SelectBackward0>)
Lyon tensor(0.1935, device='cuda:0', grad_fn=<SelectBackward0>)
Berlin tensor(0.1714, device='cuda:0', grad_fn=<SelectBackward0>)
Madrid tensor(0.1977, device='cuda:0', grad_fn=<SelectBackward0>)


Podmeos ver que realmente las probabilidades no son muy buenas. Esto es porque la capa lineal del modelo que sigue al modelo auto-regresivo, no ha sido entrenado para esta tarea. Es decir, mirando esos valores, es bastante evidente deducir que el modelo necesita ser ajustado para conseguir mejores resultados. 

### XLNet para Extractive Question Answering

Esta tarea es un poco más compleja. La entrada es una pregunta y un texto que contiene su respuesta. El modelo debe ser capaz de extraer su respuesta. 

In [16]:
from transformers import XLNetForQuestionAnsweringSimple

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForQuestionAnsweringSimple.from_pretrained("xlnet-base-cased",return_dict = True)


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForQuestionAnsweringSimple: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForQuestionAnsweringSimple were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
question = "How many continents are there in the world?"
text = "There are 7 continents in the world."

inputs = tokenizer.encode_plus(question, text, return_tensors='pt')
output = model(**inputs)

start_max = torch.argmax(F.softmax(output.start_logits, dim = -1))
end_max = torch.argmax(F.softmax(output.end_logits, dim=-1)) + 1 
## add one because of python list indexing
answer = tokenizer.decode(inputs["input_ids"][0][start_max : end_max])
print('answer: ', answer)


answer:  ?<sep> There are 7 continent


### XLNet para modelado de lenguaje

En este caso, la entrada del modelo es una oración incompleta, que el modelo debe completar. 


In [19]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased', return_dict = True)


Es necesario indicar donde está el token que debe predecir el modelo. 

In [20]:
text = "The sky is very clear at " + tokenizer.mask_token
print('origin text:', text)

# encoding the input text
input = tokenizer.encode_plus(text, return_tensors = "pt")

output = model(**input).logits[:, -1, :]
# softmax calculates a probability for each word in the XLNet's vocabulary
softmax = F.softmax(output, dim = -1)

index = torch.argmax(softmax, dim = -1)
# transform from index to word
next_word = tokenizer.decode(index)
print("precidtect word: ", next_word)

new_sentence = text.replace(tokenizer.mask_token, next_word)
print('complete sentence: ', new_sentence)

origin text: The sky is very clear at <mask>
precidtect word:  He
complete sentence:  The sky is very clear at He


El modelo no está funcionando bien, porque no ha sido entrenado para esa tarea. El modelo debe ser ajustado para esa tarea.



Es posible obtener también los mejores k candidatos propuestos por el modelo.  Para ello, usamos la función **torch.topk**

In [21]:
text = "The sky is very clear at " + tokenizer.mask_token
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
input = tokenizer.encode_plus(text, return_tensors = "pt")
output = model(**input).logits
softmax = F.softmax(output, dim = -1)
mask_word = softmax[0, mask_index, :]

top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]

for token in top_10:
  word = tokenizer.decode([token])
  new_sentence = text.replace(tokenizer.mask_token, word)
  print(new_sentence)

The sky is very clear at d
The sky is very clear at s
The sky is very clear at strongly
The sky is very clear at I
The sky is very clear at told
The sky is very clear at .
The sky is very clear at 
The sky is very clear at He
The sky is very clear at ed
The sky is very clear at heavily


###  Cómo utilizar XLNet para cualquier tarea PLN

Si quieres entrenar un modelo XLNet para una tarea específica, debes seguir los siguientes pasos:
- Cargar el modelo XLNet base.
- Agrega las capas adicionales para ajustar a la tarea (generalmente es una capa lineal).
- Finalmente, re-entrenar toda la arquitectura utilizando un conjunto de datos específico para la tarea.

La mejor forma de hacerlo es utilizar la siguiente clase de Pytorch: 

In [22]:
from transformers import XLNetModel
import torch.nn as nn

class XLNet_Model(nn.Module):
  def __init__(self, classes):
    super(XLNet_Model, self).__init__()
    self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')
    # the actual value of xlnet.config.hidden_size  is 768)
    self.out = nn.Linear(self.xlnet.config.hidden_size, classes)

  def forward(self, input):
    outputs = self.xlnet(**input)
    out = self.out(outputs.last_hidden_state)
    return out

    